In [7]:
!pip install neo4j

In [8]:
from neo4j import GraphDatabase

In [9]:
uri = 'neo4j://localhost:7687'
user = 'neo4j'
password = '1234'

In [10]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [11]:
def get_students(tx):
    query = 'MATCH (n:STUDENT) RETURN n.성명 AS 성명, n.학년 AS 학년, n.학과 AS 학과'
    result = tx.run(query)    
    for record in result:
        print(record['성명'], record['학년'], record['학과'])        

In [12]:
def run_query(func):    
    with driver.session() as session:         
        session.read_transaction(func)   

In [13]:
run_query(get_students)

이몽룡 4학년 컴퓨터공학과
성춘향 3학년 컴퓨터공학과
변사또 2학년 컴퓨터공학과


---

In [14]:
def get_name(tx):
    query = 'MATCH (a:STUDENT) WHERE a.학년 >= $grade RETURN a.성명 AS 성명'
    result = tx.run(query, grade = '3학년')    
    for record in result:
        print(record['성명'])  
        
run_query(get_name)

이몽룡
성춘향


---

In [15]:
def get_name2(tx):
    query = """
                MATCH (a:STUDENT)-[:수강하다]->(b:SUBJECT{과목명:$subject})
                RETURN a.성명 AS 성명
            """
    result = tx.run(query, subject = '딥러닝')    
    for record in result:
        print(record['성명'])  
        
run_query(get_name2)        

변사또


---

In [16]:
def get_subject(tx):
    query = """
                MATCH (a:STUDENT)-[:교제하다]->(b:STUDENT{성명:$name})
                MATCH (a) - [:수강하다] -> (d:SUBJECT)
                RETURN d.과목명 AS 과목명
            """
    result = tx.run(query, name = '성춘향')    
    for record in result:
        print(record['과목명'])  
        
run_query(get_subject) 

인공지능기초


---

In [17]:
def get_all_subject(tx):
    query = """
                MATCH (a:SUBJECT) RETURN a
            """
    result = tx.run(query)    
    for record in result:
        for node in record: 
            print(node)
            for key, val in node.items():
                print(key, ':', val)
run_query(get_all_subject) 

<Node id=3 labels=frozenset({'SUBJECT'}) properties={'과목명': '인공지능기초', '담당교수': '김교수', '학점': 2}>
과목명 : 인공지능기초
담당교수 : 김교수
학점 : 2
<Node id=4 labels=frozenset({'SUBJECT'}) properties={'과목명': '딥러닝', '담당교수': '박교수', '학점': 3}>
과목명 : 딥러닝
담당교수 : 박교수
학점 : 3


In [18]:
driver.close()